<a href="https://colab.research.google.com/github/halfnet/BacktestR/blob/master/BFA_lost_sales_prototype_run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [42]:
import pandas_gbq
import datetime
from google.cloud import bigquery
import pandas as pd
import numpy as np

In [43]:
def bleed_down_hypothetical(df):
  manufacturer_part_id = -1
  for index, row in df.iterrows():
    if row['manufacturer_part_id'] == manufacturer_part_id:
      df.at[index, 'start_qty'] = next_start_qty
      df.at[index, 'start_qty2'] = next_start_qty2
    else:
      manufacturer_part_id = row['manufacturer_part_id']
      next_start_qty = row['start_qty']
      next_start_qty2 = row['start_qty2']

    if (row['pickable_qty'] == 0) & (row['order_qty'] == 0):
      # oos and no order at all, use lost_sales to bleed down
      next_start_qty = next_start_qty + row['other_receipt_qty'] + row['hypothetical_qty'] - row['potential_lost_sales']
      next_start_qty2 = next_start_qty2 + row['other_receipt_qty'] + row['hypothetical_qty2'] - row['potential_lost_sales']
      # available qty can't be negative
      if next_start_qty < 0:
        df.at[index, 'order_qty'] = next_start_qty + row['potential_lost_sales']
        next_start_qty = 0
      else:
        df.at[index, 'order_qty'] = row['potential_lost_sales']

      if next_start_qty2 < 0:
        df.at[index, 'order_qty2'] = next_start_qty2 + row['potential_lost_sales']
        next_start_qty2 = 0
      else:
        df.at[index, 'order_qty2'] = row['potential_lost_sales']

    else:
      next_start_qty = next_start_qty + row['other_receipt_qty'] + row['hypothetical_qty'] - row['order_qty']
      next_start_qty2 = next_start_qty2 + row['other_receipt_qty'] + row['hypothetical_qty2'] - row['order_qty2']
      if next_start_qty < 0:
        df.at[index, 'order_qty'] = next_start_qty + row['order_qty']
        next_start_qty = 0

      if next_start_qty2 < 0:
        df.at[index, 'order_qty2'] = next_start_qty2 + row['order_qty2']
        next_start_qty2 = 0

  # separate hypothetical cases into two dataframes and then stack them together
  df2 = df.drop(columns=['start_qty', 'hypothetical_qty', 'order_qty'])
  df2.rename(columns={'start_qty2': 'start_qty', 'hypothetical_qty2': 'hypothetical_qty', 'order_qty2': 'order_qty'}, inplace=True)
  df2['analysis_case'] = 'hypothetical2'
  df.drop(columns=['start_qty2', 'hypothetical_qty2', 'order_qty2'], inplace=True)
  df['analysis_case'] = 'hypothetical1'
  df = pd.concat([df, df2])

  df['is_oos'] = np.where((df.start_qty + df.other_receipt_qty + df.hypothetical_qty - df.order_qty) <= 0, 1, 0)
  
  return df

In [44]:
def bleed_down_model(df):
  manufacturer_part_id = -1
  for index, row in df.iterrows():
    if row['manufacturer_part_id'] == manufacturer_part_id:
      df.at[index, 'start_qty'] = next_start_qty
    else:
      manufacturer_part_id = row['manufacturer_part_id']
      next_start_qty = row['start_qty']
    next_start_qty = next_start_qty + row['inbound_qty'] + row['proposal_qty'] - row['forecast_qty']
    next_start_qty = 0 if next_start_qty < 0 else next_start_qty

  df['is_oos'] = np.where((df.start_qty + df.inbound_qty + df.proposal_qty - df.forecast_qty) <= 0, 1, 0)

  return df

In [45]:
def create_temp_table_from_df(df):
  now = datetime.datetime.now()
  now = now.strftime("%Y%m%d%H%M%S")
  tmp_table_name = "tmp_lost_sales_bleeddown_" + now
  df.to_gbq(destination_table=dataset+"."+tmp_table_name, project_id=project_id, if_exists="replace")
  fq_tmp_table_name = "`" + project_id + "." + dataset + "." + tmp_table_name + "`"

  return tmp_table_name, fq_tmp_table_name

In [46]:
def drop_temp_table(dataset, tmp_table_name):
  sql ="""\
    DROP TABLE IF EXISTS {}.{};
  """
  query=sql.format(dataset, tmp_table_name)
  df = client.query(query).to_dataframe()

In [47]:
def prototype_run(proposal_id):

  print("-------" + str(proposal_id) + "-------")
  client = bigquery.Client(project=project_id)  

  sql ="""\
      SELECT 
        MAX(proposal.snapshot_id) snapshot_id,
        MAX(DATE(refresh.brstarttime)) refresh_date, 
        MAX(proposal.order_cadence) order_cadence,
        MAX(proposal.arrival_date) arrival_date
      FROM `wf-gcp-us-ae-ops-prod.buyfair_stream.vw_buyfair_supply_proposal` proposal
      INNER JOIN `wf-gcp-us-ae-ops-prod.buyfair_bulk.tbl_buyfair_refresh_vertica` refresh
        ON proposal.snapshot_id = refresh.brid
      WHERE proposal_id = {}
  """
  query=sql.format(proposal_id)
  df = client.query(query).to_dataframe()
  snapshot_id = df.loc[0, 'snapshot_id']
  start_date = df.loc[0, 'refresh_date']
  order_cadence = df.loc[0, 'order_cadence']
  # for now for end_date, we use later of (arrival_date + order_cadence) and today
  # we should revisit this, for example, the discontinued items should be stop at some point
  end_date = df.loc[0, 'arrival_date']
  end_date = end_date + datetime.timedelta(int(order_cadence))
  end_date = datetime.date.today() if end_date < datetime.date.today() else end_date

  ##### Delete Existing Data #####
  sql ="""\
      DELETE FROM `wf-gcp-us-ae-buyfair-prod.analytics_lost_sales.tbl_lost_sales_prototype`
      WHERE proposal_id = {}
  """
  query=sql.format(proposal_id)
  df = client.query(query).to_dataframe()

  ##### Delete Data From Previous Proposals #####
  # for hypothetical case, we need to handle deletion differently
  # this is because the start date of the simulation is not same across all products
  sql ="""\
      DELETE FROM `wf-gcp-us-ae-buyfair-prod.analytics_lost_sales.tbl_lost_sales_prototype` 
      WHERE date >= '{}'
      AND analysis_case IN ('actual', 'model')
      AND manufacturer_part_id IN
        (SELECT DISTINCT manufacturer_part_id 
        FROM `wf-gcp-us-ae-ops-prod.buyfair_stream.vw_buyfair_supply_proposal`
        WHERE proposal_id = {})
  """
  query=sql.format(start_date, proposal_id)
  df = client.query(query).to_dataframe()

  ##### Actual Case #####
  print("working on actual case...")
  sql ="""\
      DECLARE param_proposal_id INT64 DEFAULT {};
      DECLARE param_start_date DATE DEFAULT '{}';
      DECLARE param_end_date DATE DEFAULT '{}';
      CALL `wf-gcp-us-ae-buyfair-prod.analytics_lost_sales.sp_lost_sales_actual_case_data_prep`
      (param_proposal_id, param_start_date, param_end_date);
  """
  query=sql.format(proposal_id, start_date, end_date)
  df = client.query(query).to_dataframe()

  # fix some data types
  df.receipt_qty = df.receipt_qty.astype(float)
  df.demand_qty = df.demand_qty.astype(float)
  df.wholesale_cost_per_unit = df.wholesale_cost_per_unit.astype(float)
  df.revenue_per_unit = df.revenue_per_unit.astype(float)
  df.date = pd.to_datetime(df.date)

  df = df.sort_values(by=['manufacturer_part_id','date'])
  # fill in missing values using linear interpolation
  df.pickable_qty = df.pickable_qty.interpolate(limit_direction='both')
  df.wholesale_cost_per_unit = df.wholesale_cost_per_unit.interpolate(limit_direction='both')
  df.revenue_per_unit = df.revenue_per_unit.interpolate(limit_direction='both')
  df['is_oos'] = np.where(df.pickable_qty <= 0, 1, 0)

  tmp_table_name, fq_tmp_table_name = create_temp_table_from_df(df)

  # insert data to real table
  sql ="""\
      DECLARE param_fg_tmp_table_name STRING DEFAULT '{}';
      CALL `wf-gcp-us-ae-buyfair-prod.analytics_lost_sales.sp_lost_sales_actual_case_data_insert`(param_fg_tmp_table_name);
  """
  query=sql.format(fq_tmp_table_name)
  df = client.query(query).to_dataframe()

  drop_temp_table(dataset, tmp_table_name)

  ##### Hypothetical Case #####
  print("working on hypothetical case...")
  sql ="""\
      DECLARE param_proposal_id INT64 DEFAULT {};
      DECLARE param_start_date DATE DEFAULT '{}';
      DECLARE param_end_date DATE DEFAULT '{}';
      CALL `wf-gcp-us-ae-buyfair-prod.analytics_lost_sales.sp_lost_sales_hypothetical_case_data_prep`
      (param_proposal_id, param_start_date, param_end_date);
  """
  query=sql.format(proposal_id, start_date, end_date)
  df = client.query(query).to_dataframe()

  # the following columns are duplicated to support hypothetical case 1 and 2
  #     start_qty
  #     order_qty
  #     hypothetical_qty
  # fix some data types
  df.start_qty = df.start_qty.astype(float)
  df['start_qty2'] = df.start_qty
  df.other_receipt_qty = df.other_receipt_qty.astype(float)
  df.order_qty = df.order_qty.astype(float)
  df['order_qty2'] = df.order_qty
  df.wholesale_cost_per_unit = df.wholesale_cost_per_unit.astype(float)
  df.revenue_per_unit = df.revenue_per_unit.astype(float)
  df.date = pd.to_datetime(df.date)
  df.pickable_qty = df.pickable_qty.astype(float)
  df.on_hand_qty = df.on_hand_qty.astype(float)

  # fill missing data
  df.pickable_qty = df.pickable_qty.interpolate(limit_direction='both')
  df.on_hand_qty = df.on_hand_qty.interpolate(limit_direction='both')
  df.wholesale_cost_per_unit = df.wholesale_cost_per_unit.interpolate(limit_direction='both')
  df.revenue_per_unit = df.revenue_per_unit.interpolate(limit_direction='both')

  # if products proposed but are not ordered or not arrived, simply use proposed qty and date it suppose to arrive
  # once arrived, we need to distribute the proposed qty based on actual receipt qtys and dates
  df['hypothetical_qty'] = np.where(df.proposal_receipt_total_qty == 0, df.proposal_qty, 
                                    1.0 * df.proposal_l1_qty * df.proposal_receipt_qty / df.proposal_receipt_total_qty)
  df['hypothetical_qty2'] = np.where(df.proposal_receipt_total_qty == 0, df.proposal_qty2, 
                                    1.0 * df.proposal_l2_qty * df.proposal_receipt_qty / df.proposal_receipt_total_qty)

  df.drop(columns=['proposal_receipt_qty', 'proposal_qty', 'proposal_qty2', 'proposal_l1_qty', 'proposal_l2_qty',
                  'proposal_l4_qty', 'proposal_receipt_total_qty'], inplace=True)

  # now let's separate the products into 2 groups (dataframes)
  # 1 - products that don't exist before --> bleed down from refresh date
  # 2 - products that do exist --> bleed down from impact date (this can be receipt date or expected arrival date)
  #                                 then delete duplicated records
  df_new = df[df.existing_product == 0].copy()
  df_new.drop(columns=['on_hand_qty'], inplace=True)
  df_new = bleed_down_hypothetical(df_new)

  df_exist = df[df.existing_product == 1].copy()
  # for each product, find the first date it is received
  # backout a couple of days from the pickable date
  df_exist_product = df_exist[df_exist.hypothetical_qty>0].groupby(['manufacturer_part_id'], as_index=False).agg(
      {'date': 'min'})
  df_exist_product.date = df_exist_product.date + datetime.timedelta(-2)
  df_exist_product.rename(columns={'date': 'impact_date'}, inplace=True)
  df_exist = pd.merge(df_exist,
                      df_exist_product,
                      left_on='manufacturer_part_id',
                      right_on='manufacturer_part_id',
                      how='inner')
  df_exist = df_exist[df_exist.date >= df_exist.impact_date].copy()
  # now initialize the start qty with on_hand_qty (note: not pickable_qty)
  df_exist.start_qty = np.where(df_exist.date == df_exist.impact_date, df_exist.on_hand_qty, 0)
  df_exist.start_qty2 = df_exist.start_qty
  df_exist.drop(columns=['on_hand_qty', 'impact_date'], inplace=True)
  df_exist = bleed_down_hypothetical(df_exist)

  df = pd.concat([df_new, df_exist])

  tmp_table_name, fq_tmp_table_name = create_temp_table_from_df(df)

  # now delete older records basing on what we are about to insert
  sql ="""\
      DELETE FROM `wf-gcp-us-ae-buyfair-prod.analytics_lost_sales.tbl_lost_sales_prototype` pt 
      WHERE date >= '{}'
      AND EXISTS(SELECT 1 FROM {} tmp
        WHERE pt.analysis_case = tmp.analysis_case AND pt.date = CAST(tmp.date AS DATE) AND pt.manufacturer_part_id = tmp.manufacturer_part_id)
  """
  query=sql.format(start_date, fq_tmp_table_name)
  df = client.query(query).to_dataframe()

  # insert data to real table
  sql ="""\
      DECLARE param_fg_tmp_table_name STRING DEFAULT '{}';
      CALL `wf-gcp-us-ae-buyfair-prod.analytics_lost_sales.sp_lost_sales_hypothetical_case_data_insert`(param_fg_tmp_table_name);
  """
  query=sql.format(fq_tmp_table_name)
  df = client.query(query).to_dataframe()

  drop_temp_table(dataset, tmp_table_name)

  ###### Model Case #####
  print("working on model case...")
  sql ="""\
      DECLARE param_snapshot_id INT64 DEFAULT {};
      DECLARE param_proposal_id INT64 DEFAULT {};
      DECLARE param_start_date DATE DEFAULT '{}';
      DECLARE param_end_date DATE DEFAULT '{}';
      CALL `wf-gcp-us-ae-buyfair-prod.analytics_lost_sales.sp_lost_sales_model_case_data_prep`
      (param_snapshot_id, param_proposal_id, param_start_date, param_end_date);
  """

  query=sql.format(snapshot_id, proposal_id, start_date, end_date)
  df = client.query(query).to_dataframe()

  # fix some data types
  df.start_qty = df.start_qty.astype(float)
  df.inbound_qty = df.inbound_qty.astype(float)
  df.proposal_qty = df.proposal_qty.astype(float)
  df.forecast_qty = df.forecast_qty.astype(float)
  df.date = pd.to_datetime(df.date)
  df.wholesale_cost_per_unit = df.wholesale_cost_per_unit.astype(float)
  df.revenue_per_unit = df.revenue_per_unit.astype(float)
  df.potential_lost_sales = df.potential_lost_sales.astype(float)
  df.wholesale_cost_per_unit = df.wholesale_cost_per_unit.interpolate(limit_direction='both')
  df.revenue_per_unit = df.revenue_per_unit.interpolate(limit_direction='both')

  df = bleed_down_model(df)

  tmp_table_name, fq_tmp_table_name = create_temp_table_from_df(df)

  # insert data to real table
  sql ="""\
      DECLARE param_fg_tmp_table_name STRING DEFAULT '{}';
      CALL `wf-gcp-us-ae-buyfair-prod.analytics_lost_sales.sp_lost_sales_model_case_data_insert`(param_fg_tmp_table_name);
  """
  query=sql.format(fq_tmp_table_name)
  df = client.query(query).to_dataframe()

  drop_temp_table(dataset, tmp_table_name)


In [48]:
begin_time = datetime.datetime.now()

In [49]:
project_id = "wf-gcp-us-ae-buyfair-prod"
dataset = "analytics_lost_sales"

In [50]:
client = bigquery.Client(project=project_id)  
sql ="""\
    SELECT *
    FROM `wf-gcp-us-ae-ops-prod.buyfair_stream.vw_buyfair_supply_proposal_used`
    ORDER BY snapshot_id
"""
query=sql.format()
df = client.query(query).to_dataframe()
proposals = df.proposal_id.tolist()

In [51]:
# change the proposal id and run the script by Ctrl+F10
#proposals = [5799107, 5952237]
#proposals = [5801358, 5904781]
#proposals = [5756686, 5850428]
#proposals = [5852230, 5956819, 6099142]
#proposals = [6099142]

In [52]:
for proposal_id in proposals:
  prototype_run(proposal_id)

-------5852230-------
working on actual case...


1it [00:03,  3.84s/it]


working on hypothetical case...


1it [00:02,  2.17s/it]


working on model case...


...

1it [00:02,  2.97s/it]


working on hypothetical case...


1it [00:02,  2.28s/it]


working on model case...


1it [00:04,  4.45s/it]


In [53]:
print(datetime.datetime.now() - begin_time)

0:12:33.218363
